# 4.2 Prokaryote antiphage defence

## Software and versions used in this study

- CRISPRDetect v2.4
- PADLOC v2.0.0 (database: PADLOC-DB v2.0.0)

## Additional custom scripts

Note: custom scripts have been tested in python v3.11.6 and R v4.2.1 and may not be stable in other versions.

- scripts/general/antiphage_padloc_summary_table.py

*Required python packages: argparse, pandas, numpy, re, glob*

***

## Antiphage defence mechanisms

Identify antiphage defence mechanisms present in prokaryote MAGs data via PADLOC and CRISPRDetect

#### Run CRISPRDetect

Note: It may be necessary to edit the header 'ArrayQualityScore' output by CRISPRDetect to be 'Array_quality_score' (e.g. this is the format expected by PADLOC v2.0.0)

In [ ]:
mkdir -p DNA/2.prokaryote_mags/6.antiphage_defense/crispr_detect

for mag_fna in DNA/2.prokaryote_mags/4.bin_dereplication_dRep/1.dRep_out/dereplicated_genomes/*.fa; do
    output_name=$(basename ${mag_fna} .fa)
    perl CRISPRDetect.pl -array_quality_score_cutoff 1 -T 16 \
    -f ${mag_fna} \
    -o DNA/2.prokaryote_mags/6.antiphage_defense/crispr_detect/${output_name} \
    > DNA/2.prokaryote_mags/6.antiphage_defense/crispr_detect/${output_name}.log 
    # Edit 'ArrayQualityScore' to 'Array_quality_score' (formatting expected by this version of PADLOC)
    sed -i -e "s/ArrayQualityScore/Array_quality_score/g" DNA/2.prokaryote_mags/6.antiphage_defense/crispr_detect/${output_name}.gff
done

#### Run PADLOC

In [ ]:
mkdir -p DNA/2.prokaryote_mags/6.antiphage_defense/padloc

for mag_fna in DNA/2.prokaryote_mags/4.bin_dereplication_dRep/1.dRep_out/dereplicated_genomes/*.fa; do
    output_name=$(basename ${mag_fna} .fa)
    padloc --cpu 16 --force \
    --fna ${mag_fna} \
    --crispr DNA/2.prokaryote_mags/6.antiphage_defense/crispr_detect/${output_name}.gff \
    --outdir DNA/2.prokaryote_mags/6.antiphage_defense/padloc
done


#### Concatenate results into summary table via *antiphage_padloc_summary_table.py*

In [ ]:
scripts/general/antiphage_padloc_summary_table.py \
-p DNA/2.prokaryote_mags/6.antiphage_defense/padloc/ \
-o DNA/2.prokaryote_mags/6.antiphage_defense/padloc.summary_table.csv

***